In [8]:
from dotenv import load_dotenv
load_dotenv()
import os

In [11]:
import openai
import json

# Sample Vietnamese product text
text = """💻MacBook Pro - Giá: 5.5 triệu

🔥 Cấu hình : Chip i5, ram 8G, SSD 256G ,máy 99%

/~Li Máy chính hãng zin 100%
/~Li Giá sỉ TOÀN QUỐC siêu rê
/~Li Máy chọn đẹp 99%, bảo hành
/~Li Màn đẹp, loa to, máy nhanh
/~Li Free ship Toàn Quốc + 4 Quà
/~Li Đổi tuỳ ý 15 ngày đầu Free
📞 Hotline : 0982.729.830"""

# Template for extraction
template = """
# Task: Rút trích thông tin chi tiết về sản phẩm máy tính từ mẫu quảng cáo.
Bao gồm các thông tin sau:
- Tên sản phẩm
- Giá
- Cấu hình máy
- Hotline hoặc thông tin liên hệ

# Steps:
1. Xác định tên sản phẩm.
2. Xác định giá sản phẩm.
3. Xác định các thông tin về cấu hình máy (chip, RAM, SSD, tình trạng máy, v.v.).
4. Trích xuất thông tin liên hệ (số điện thoại hotline).

# Output format:
{
"product_name": "<tên sản phẩm>",
"price": "<giá sản phẩm>",
"features": "<cấu hình máy>",
"hotline": "<điện thoại liên hệ>",
}

# Input:
{{text}}
"""

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def extract_vietnamese_product_info(text):
    """
    Extract product information from Vietnamese text using the template.
    """
    # Create the prompt by replacing {{text}} with actual text
    prompt = template.replace("{{text}}", text)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that extracts product information from Vietnamese product advertisements. Always respond with valid JSON."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format={"type": "json_object"}
    )

    # Print the raw response for debugging
    print("Raw response content:", response.choices[0].message.content)
    # Parse the JSON response
    result = json.loads(response.choices[0].message.content)
    return result


# Alternative: Using structured output with JSON schema
def extract_with_schema(text):
    """
    Extract Vietnamese product information using structured outputs.
    """
    prompt = f"""Rút trích thông tin chi tiết về sản phẩm máy tính từ mẫu quảng cáo sau:

{text}

Bao gồm: tên sản phẩm, giá, cấu hình máy, và hotline."""

    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": "Bạn là trợ lý trích xuất thông tin sản phẩm từ quảng cáo tiếng Việt."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "product_extraction",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "product_name": {
                            "type": "string",
                            "description": "Tên sản phẩm"
                        },
                        "price": {
                            "type": "string",
                            "description": "Giá sản phẩm"
                        },
                        "features": {
                            "type": "string",
                            "description": "Thông tin cấu hình máy"
                        },
                        "hotline": {
                            "type": "string",
                            "description": "Số điện thoại liên hệ"
                        }
                    },
                    "required": ["product_name", "price", "features", "hotline"],
                    "additionalProperties": False
                }
            }
        }
    )

    result = json.loads(response.choices[0].message.content)
    return result


# Example usage
if __name__ == "__main__":
    print("Vietnamese Product Information Extraction")
    print("=" * 60)
    print(f"\nInput Text:\n{text}\n")
    print("-" * 60)

    try:
        # Method 1: Template-based
        print("\nMethod 1: Template-based extraction")
        result1 = extract_vietnamese_product_info(text)
        print(json.dumps(result1, indent=2, ensure_ascii=False))

        print("\n" + "-" * 60)

        # Method 2: Structured output
        print("\nMethod 2: Structured output extraction")
        result2 = extract_with_schema(text)
        print(json.dumps(result2, indent=2, ensure_ascii=False))

    except Exception as e:
        print(f"Error: {str(e)}")
        print("\nNote: Make sure to replace 'your-api-key-here' with your actual OpenAI API key")

Vietnamese Product Information Extraction

Input Text:
💻MacBook Pro - Giá: 5.5 triệu

🔥 Cấu hình : Chip i5, ram 8G, SSD 256G ,máy 99%

/~Li Máy chính hãng zin 100%
/~Li Giá sỉ TOÀN QUỐC siêu rê
/~Li Máy chọn đẹp 99%, bảo hành
/~Li Màn đẹp, loa to, máy nhanh
/~Li Free ship Toàn Quốc + 4 Quà
/~Li Đổi tuỳ ý 15 ngày đầu Free
📞 Hotline : 0982.729.830

------------------------------------------------------------

Method 1: Template-based extraction
Raw response content: {
  "product_name": "MacBook Pro",
  "price": "5.5 triệu",
  "features": "Chip i5, ram 8G, SSD 256G, máy 99%",
  "hotline": "0982.729.830"
}
{
  "product_name": "MacBook Pro",
  "price": "5.5 triệu",
  "features": "Chip i5, ram 8G, SSD 256G, máy 99%",
  "hotline": "0982.729.830"
}

------------------------------------------------------------

Method 2: Structured output extraction
{
  "product_name": "MacBook Pro",
  "price": "5.5 triệu",
  "features": "Chip i5, ram 8G, SSD 256G ,máy 99%",
  "hotline": "0982.729.830"
}
